## Setup for TBMaLT examples

In [ ]:
import os
from os.path import exists, join
import urllib, tempfile, tarfile, zipfile

import torch

from tbmalt.io.skf import Skf
from tbmalt.tools.downloaders import download_dftb_parameter_set
torch.set_default_dtype(torch.float64)

## Download auorg-1-1 parameter set

In [ ]:
# Link to the auorg-1-1 parameter set
link = 'https://github.com/dftbparams/auorg/releases/download/v1.1.0/auorg-1-1.tar.xz'

# Path of the parameter set
try:
    os.mkdir("./data")
except FileExistsError:
    pass
    
parameter_db_path = "./data/example_dftb_parameters.h5"

if not exists(parameter_db_path):
    download_dftb_parameter_set(link, parameter_db_path)

## Download data for dos and vcr training

In [ ]:
# Link to the dos training data
link = 'https://zenodo.org/records/15592694/files/tbmalt_data.zip?download=1'

# Path of the data
output_file = "./data/dos.zip"

# Download the data
req = urllib.request.Request(link)
with urllib.request.urlopen(req) as response, open(output_file, 'wb') as out_file:
    out_file.write(response.read())

# Extract the data
with zipfile.ZipFile(output_file, 'r') as zip_ref:
    zip_ref.extractall("./data/")

os.remove(output_file)